In [1]:
import pandas as pd
import numpy as np

In [2]:
# Importing Dataset
df = pd.read_excel("New_Dataset.xlsx")

In [3]:
import re
R_patterns = [
   (r'won\'t', 'will not'),
   (r'can\'t', 'can not'),
   (r'(\w+)\'m', '\g<1> am'),
   (r'(\w+)\'ll', '\g<1> will'),
   (r'(\w+)\'d like to', '\g<1> would like to'),
   (r'(\w+)n\'t', '\g<1> not'),
   (r'(\w+)\'ve', '\g<1> have'),
   (r'(\w+)\'s', '\g<1> is'),
   (r'(\w+)\'re', '\g<1> are')
]

class REReplacer(object):
   def __init__(self, patterns = R_patterns):
      self.patterns = [(re.compile(regex), repl) for (regex, repl) in R_patterns]
   def replace(self, text):
      s = text
      for (pattern, repl) in self.patterns:
         s = re.sub(pattern, repl, s)
      return s

In [4]:
rep_word = REReplacer()

def text_lower(x):
    return x.lower()
def corrector(x):
    return x.replace('. ', '.')

df['Model Answer'] = df['Model Answer'].apply(rep_word.replace)
df['User Answer'] = df['User Answer'].apply(rep_word.replace)

mod_ans_set = df.iloc[:, 2].values
usr_ans_set = df.iloc[:, 3].values

df['Model Answer'] = df['Model Answer'].apply(text_lower)
df['User Answer'] = df['User Answer'].apply(text_lower)

df['Model Answer'] = df['Model Answer'].apply(corrector)
df['User Answer'] = df['User Answer'].apply(corrector)

In [5]:
def replace_punct(x):
    pattern = "[^\w\d\+\*\-\\\=\s]"   
    repl = " "
    
    return re.sub(pattern, repl, x)

def remove_extra(text):
    return " ".join(text.strip().split())

df['Model Answer'] = df['Model Answer'].apply(replace_punct)
df['User Answer'] = df['User Answer'].apply(replace_punct)

df['Model Answer'] = df['Model Answer'].apply(remove_extra)
df['User Answer'] = df['User Answer'].apply(remove_extra)

In [6]:
from keybert import KeyBERT
import yake

def keybert_keywordExtractor1(text):
    kw_model = KeyBERT()
    return kw_model.extract_keywords(text, top_n = 50)
    
def keybert_keywordExtractor2(text):
    kw_model = KeyBERT()
    return kw_model.extract_keywords(text, top_n = 101)
def yake_keywordExtractor(text, language = "en", max_ngram_size = 1, deduplication_threshold = 0.9, deduplication_algo = 'seqm', windowSize = 2, numOfKeywords = 101):
    
    custom_kw_extractor = yake.KeywordExtractor(lan = language, 
                                            n = max_ngram_size, 
                                            dedupLim = deduplication_threshold, 
                                            dedupFunc = deduplication_algo, 
                                            windowsSize = windowSize, 
                                            top = numOfKeywords, 
                                            features = None)
    
    return custom_kw_extractor.extract_keywords(text)

c:\Users\harsh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df['keybert_keys_sentence1'] = df['Model Answer'].apply(keybert_keywordExtractor1)
df['keybert_keys_sentence2'] = df['User Answer'].apply(keybert_keywordExtractor2)

df['yake_keys_sentence1'] = df['Model Answer'].apply(yake_keywordExtractor)
df['yake_keys_sentence2'] = df['User Answer'].apply(yake_keywordExtractor)

In [8]:
import en_core_web_trf
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.corpus import wordnet

def scoring_unit(keywords_text1, keywords_text2):
    match = 0
    total = 0
    synonym_dict = []
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    for token in keywords_text1:
        total += token[1]
    
    if(total == 0):
        return 10
    
    for var in keywords_text2:
        syn = wordnet.synsets(var[0])
        syn_words = [x.lemma_names() for x in syn]
        syn_words = [x for elem in syn_words for x in elem]
        syn_words.append(var[0])
        syn_words = list(set(syn_words))
        #print(syn_words)
        temp = []
        wt = word_tokenize(var[0])
        pos = pos_tag(wt)[0][1]
        for i in range(0, len(syn_words)):
            checker_wt = word_tokenize(syn_words[i])
            checker_pos = pos_tag(wt)[0][1]
            if(pos == checker_pos):
                temp.append(syn_words[i])
        synonym_dict = synonym_dict + temp
    #print(synonym_dict)
    
    for token in keywords_text1:
        syn = wordnet.synsets(token[0])
        syn_words = [x.lemma_names() for x in syn]
        syn_words = [x for elem in syn_words for x in elem]
        syn_words.append(token[0])
        syn_words = list(set(syn_words))
        if len(set(syn_words).intersection(set(synonym_dict)))!=0:
            # print(token[0])
            match += token[1]
            
    return ((match*10)/total)

In [9]:
df['keybert_score'] = df[['keybert_keys_sentence1', 'keybert_keys_sentence2']].apply(lambda x:scoring_unit(
    x.keybert_keys_sentence1, 
    x.keybert_keys_sentence2),
    axis=1
    )

df['yake_score'] = df[['yake_keys_sentence1', 'yake_keys_sentence2']].apply(lambda x:scoring_unit(
    x.yake_keys_sentence1, 
    x.yake_keys_sentence2), 
    axis=1
    )

In [11]:
df.drop(['keybert_keys_sentence1', 'keybert_keys_sentence2', 'yake_keys_sentence1',	'yake_keys_sentence2'], axis=1)

,S. No,Question,Model Answer,User Answer,Actual Score (0-10),keybert_score,yake_score
0,1,What is an Operating Systems?Why do we need an...,an operating system os is a collection of soft...,an operating system is a program that acts as ...,6,7.928716,6.580460
1,2,Explain different types of tasks done by OS an...,operating system services and facilites can be...,following are the services provided by an oper...,5,5.903012,5.158648
2,3,Give the view of OS as a Resource Manager,the concept of an operating system as providin...,from a high-level perspective an operating sys...,9,7.433753,6.220559
3,4,Give the view of OS as an extended machine.,the operating systems perform two basically un...,from another perspective an operating system o...,5,5.428903,4.514964
4,5,What is Deadlock Avoidance?,deadlock avoidance is a set of techniques used...,deadlock avoidance is a technique used in oper...,5,8.288505,6.202614
...,...,...,...,...,...,...,...
75,76,What is question answering system?,question-answering qa systems are computer sof...,question-answering qa systems are computer sof...,7,7.116238,6.525868
76,77,What is question answering system?,question-answering qa systems are computer sof...,question-answering qa systems are computer sof...,6,6.446863,5.720182
77,78,What is deadlock ignorance?,deadlock is also known as the ostrich algorith...,deadlock is also known as the ostrich algorith...,9,9.700411,9.677047
78,79,What is deadlock? Explain necessary and suffic...,a deadlock in os is a situation in which more ...,a deadlock in os is a situation in which more ...,9,9.801265,9.668062


In [12]:
num_of_inputs = len(mod_ans_set)
sim_score = [0]*num_of_inputs
sbert_score = [0]*num_of_inputs
hf_score = [0] * num_of_inputs

In [13]:
from nltk.tokenize import sent_tokenize
from numpy.linalg import norm

def cos_sim(sent1_emb, sent2_emb):
  cos = np.dot(sent1_emb, sent2_emb)/(norm(sent1_emb)*norm(sent2_emb))
  return cos

def similarity_score(text1, text2, model):
  mod_sent = sent_tokenize(text1)
  usr_sent = sent_tokenize(text2)

  mod_emb = []
  usr_emb = []

  for sent in mod_sent:
    sent_emb = model.encode(sent)
    mod_emb.append(sent_emb)
  
  for sent in usr_sent:
    sent_emb = model.encode(sent)
    usr_emb.append(sent_emb)

  n = len(mod_sent)
  m = len(usr_sent)

  sim_ans = 0

  for i in range(0, n):
    for j in range(0, m):
      if(cos_sim(mod_emb[i], usr_emb[j]) >= 0.7):
        sim_ans += 1
        break
  
  sim_ans /= n
  return sim_ans

In [16]:
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import torch

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

def summarize(text):
  preprocessed_text = text.strip().replace('\n', '')
  t5_input_text = 'summarize: ' + preprocessed_text

  tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)

  summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  return summary

c:\Users\harsh\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\harsh\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
from sentence_transformers import CrossEncoder
model_simcse = SentenceTransformer('princeton-nlp/sup-simcse-roberta-large')
model_crossencoder = CrossEncoder('cross-encoder/stsb-roberta-base')
model_sbert = SentenceTransformer('stsb-mpnet-base-v2')
model_hf = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

No sentence-transformers model found with name C:\Users\harsh/.cache\torch\sentence_transformers\princeton-nlp_sup-simcse-roberta-large. Creating a new one with MEAN pooling.


In [20]:
for i in range(0, num_of_inputs):
    mod_ans = mod_ans_set[i]
    usr_ans = usr_ans_set[i]

    usr_ans += summarize(usr_ans)
    
    sim_val = similarity_score(mod_ans, usr_ans, model_simcse)
    sim_score[i] = sim_val
    sbert_val = similarity_score(mod_ans, usr_ans, model_sbert)
    sbert_score[i] = sbert_val
    hf_val = similarity_score(mod_ans, usr_ans, model_hf)
    hf_score[i] = hf_val

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [21]:
simcse_score = sim_score
sbert_scoring = sbert_score
hf_score_normalized = hf_score
for i in range(0, num_of_inputs):
  simcse_score[i] = sim_score[i] * 10
  sbert_scoring[i] = sbert_scoring[i] * 10
  hf_score_normalized[i] = hf_score[i] * 10

In [22]:
simcse_score = np.array(simcse_score)
sbert_scoring = np.array(sbert_scoring)
hf_score_normalized = np.array(hf_score_normalized)

df['Sim_CSE Score'] = simcse_score.tolist()
df['SBERT Score'] = sbert_scoring.tolist()
df['HF_Score'] = hf_score_normalized.tolist()

In [23]:
df.head()

,S. No,Question,Model Answer,User Answer,Actual Score (0-10),keybert_keys_sentence1,keybert_keys_sentence2,yake_keys_sentence1,yake_keys_sentence2,keybert_score,yake_score,Sim_CSE Score,SBERT Score,HF_Score
0,1,What is an Operating Systems?Why do we need an...,an operating system os is a collection of soft...,an operating system is a program that acts as ...,6,"[(os, 0.5709), (operating, 0.3746), (software,...","[(os, 0.5634), (operating, 0.4578), (software,...","[(computer, 0.014082021907517523), (operating,...","[(computer, 0.01460579215540396), (operating, ...",7.928716,6.580460,8.750000,3.750000,5.000000
1,2,Explain different types of tasks done by OS an...,operating system services and facilites can be...,following are the services provided by an oper...,5,"[(os, 0.477), (programs, 0.4362), (operating, ...","[(operating, 0.5053), (security, 0.4239), (exe...","[(system, 0.01484175578162468), (operating, 0....","[(output, 0.03937187525108959), (device, 0.040...",5.903012,5.158648,9.090909,0.909091,1.818182
2,3,Give the view of OS as a Resource Manager,the concept of an operating system as providin...,from a high-level perspective an operating sys...,9,"[(os, 0.4752), (operating, 0.4153), (programs,...","[(os, 0.4744), (processes, 0.4268), (operating...","[(system, 0.014646037261204945), (operating, 0...","[(resources, 0.013877918771042804), (memory, 0...",7.433753,6.220559,10.000000,3.750000,1.250000
3,4,Give the view of OS as an extended machine.,the operating systems perform two basically un...,from another perspective an operating system o...,5,"[(operating, 0.4447), (computers, 0.4103), (st...","[(os, 0.5087), (operating, 0.371), (systems, 0...","[(operating, 0.017152339201577816), (system, 0...","[(system, 0.010976070087257307), (machine, 0.0...",5.428903,4.514964,5.000000,1.000000,1.000000
4,5,What is Deadlock Avoidance?,deadlock avoidance is a set of techniques used...,deadlock avoidance is a technique used in oper...,5,"[(deadlocks, 0.5929), (deadlock, 0.5799), (pro...","[(deadlocks, 0.5647), (deadlock, 0.5587), (pro...","[(deadlock, 0.013010677940705515), (system, 0....","[(process, 0.017118988225377267), (deadlock, 0...",8.288505,6.202614,7.142857,2.857143,4.285714


In [24]:
df = df.drop(['keybert_keys_sentence1', 'keybert_keys_sentence2', 'yake_keys_sentence1',	'yake_keys_sentence2'], axis=1)


In [25]:
df.head()

,S. No,Question,Model Answer,User Answer,Actual Score (0-10),keybert_score,yake_score,Sim_CSE Score,SBERT Score,HF_Score
0,1,What is an Operating Systems?Why do we need an...,an operating system os is a collection of soft...,an operating system is a program that acts as ...,6,7.928716,6.580460,8.750000,3.750000,5.000000
1,2,Explain different types of tasks done by OS an...,operating system services and facilites can be...,following are the services provided by an oper...,5,5.903012,5.158648,9.090909,0.909091,1.818182
2,3,Give the view of OS as a Resource Manager,the concept of an operating system as providin...,from a high-level perspective an operating sys...,9,7.433753,6.220559,10.000000,3.750000,1.250000
3,4,Give the view of OS as an extended machine.,the operating systems perform two basically un...,from another perspective an operating system o...,5,5.428903,4.514964,5.000000,1.000000,1.000000
4,5,What is Deadlock Avoidance?,deadlock avoidance is a set of techniques used...,deadlock avoidance is a technique used in oper...,5,8.288505,6.202614,7.142857,2.857143,4.285714


In [26]:
df.to_excel("New_Dataset_Graded.xlsx")